# Content beased filtering using BERT
Recommendations using BERT

In [10]:
import pandas as pd
import numpy as np

import re
import string

from nltk.classify.textcat import TextCat

import dask.dataframe as dd
import multiprocessing

from keybert import KeyBERT

from sklearn.feature_extraction.text import TfidfVectorizer

import matplotlib.pyplot as plt
import altair as alt
alt.renderers.enable('mimetype')

from sklearn.metrics.pairwise import cosine_similarity

### Data Cleaning

In [ ]:
path='./Data/'

df_books1=pd.read_csv(path +'Books-old.csv') 
df_books2 = pd.read_csv(path + '7k-Books.csv') #To skip bad lines
df_ratings=pd.read_csv(path + 'Ratings.csv') 
df_users=pd.read_csv(path + 'Users.csv') 

#Renameing
df_books2.rename(columns={'isbn10':'ISBN'}, inplace=True)


C:\Users\Lars\AppData\Local\Temp\ipykernel_23068\402415562.py:3: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_books1=pd.read_csv(path +'Books-old.csv')


In [30]:
print("df_books1:")
print(f"Shape: {df_books1.shape}")
print(f"Columns: {df_books1.columns.tolist()}\n")

print("df_books2:")
print(f"Shape: {df_books2.shape}")
print(f"Columns: {df_books2.columns.tolist()}")

df_books1:
Shape: (271360, 8)
Columns: ['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher', 'Image-URL-S', 'Image-URL-M', 'Image-URL-L']

df_books2:
Shape: (6810, 12)
Columns: ['isbn13', 'ISBN', 'title', 'subtitle', 'authors', 'categories', 'thumbnail', 'description', 'published_year', 'average_rating', 'num_pages', 'ratings_count']


In [31]:
# Merged dataset



merged_book_df=pd.merge(df_books1, df_books2, on='ISBN', how='inner') 

print("df_books_merged:")
print(f"Shape: {merged_book_df.shape}")
print(f"Columns: {merged_book_df.columns.tolist()}")



merged_book_df.dropna(subset=["description"], inplace=True)

print("df_books_merged:")
print(f"Shape: {merged_book_df.shape}")
print(f"Columns: {merged_book_df.columns.tolist()}")


df_books_merged:
Shape: (2447, 19)
Columns: ['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher', 'Image-URL-S', 'Image-URL-M', 'Image-URL-L', 'isbn13', 'title', 'subtitle', 'authors', 'categories', 'thumbnail', 'description', 'published_year', 'average_rating', 'num_pages', 'ratings_count']
df_books_merged:
Shape: (2369, 19)
Columns: ['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher', 'Image-URL-S', 'Image-URL-M', 'Image-URL-L', 'isbn13', 'title', 'subtitle', 'authors', 'categories', 'thumbnail', 'description', 'published_year', 'average_rating', 'num_pages', 'ratings_count']
